In [1]:
%load_ext autoreload
%autoreload 2
from model import create_dpnet_one
model = create_dpnet_one(in_channel=18)

==> Loading pretrained model model/pretrained_models/hrnetv2_w48_imagenet_pretrained.pth


In [2]:
import torch
x = torch.rand((5,18,256,256))
outputs = model(x)

In [3]:
%load_ext autoreload
%autoreload 2

from torchvision import transforms
from model import dataloader
import pandas as pd
import os

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

data_dir = '/meladyfs/newyork/loctrinh/DATASETS/'
frame_count = {'FF++': pd.read_csv(os.path.join(data_dir, 'FF++', 'video_stat.csv'), index_col=0)}
train_df = pd.read_csv(os.path.join(data_dir, 'FF++/splits/{}_trainlist_01.csv'.format('Deepfakes')))
val_df = pd.read_csv(os.path.join(data_dir, 'FF++/splits/{}_vallist_01.csv'.format('Deepfakes')))
test_df = pd.read_csv(os.path.join(data_dir, 'FF++/splits/{}_testlist_01.csv'.format('Deepfakes')))

data_loader = dataloader.StackNImageLoader(10, 2, data_dir, frame_count, train_df, val_df, test_df, train_transform, test_transform)
train_loader, val_loader, test_loader, push_loader = data_loader.run()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
==> Training data: 388800 frames
==> Validation data: 28000 frames
==> Testing data: 28000 frames
==> Pushing data: 144000 frames


In [4]:
%%timeit
imgs, labels = next(iter(train_loader))

4.39 s ± 344 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
video_name, frame_index, imgs, labels = next(iter(push_loader))

In [ ]:
from utils import *
from PIL import Image
idx = 3
print(labels[idx])
unform = UnNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
display(Image.fromarray(tensor2uint8(unform(imgs[idx][:3]))))
display(Image.fromarray(tensor2uint8(imgs[idx][4:5])))